# ML Gdansk (1 VI 2020)

In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

data_path = "data_location3"
sys.path.insert(0, "/home/jupyter/cache_disk2/MLG1VI2020")
from ds_utils_MLG import *

print(f'TensorFlow version: {tf.__version__}')

In [ ]:
TARGET_SIZE = (224,224,3)
N_Classes = 196

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Activation, Lambda, Flatten
from tensorflow.keras.models import Model, load_model

### 1. Download and prepare dataset (make TFRecords)

In [ ]:
(ds_train, ds_valid, ds_test), ds_info = tfds.load(
    'cars196',    
    split=['train', 'test[50%:]', 'test[:50%]'],    
    as_supervised=True,
    with_info=True)

In [ ]:
ds_info

See what we got:

In [ ]:
display_images_and_labels_from_dataset(dataset=ds_train)

In [ ]:
def get_filtered_resized_dataset(dataset, filter_label_value):
    dataset = dataset.filter(lambda image, label: tf.equal(label, filter_label_value))
    return dataset.map(lambda image, label: (tf.cast(tf.image.resize(image, TARGET_SIZE[:-1]),tf.uint8), label))    

In [ ]:
some_filtered_ds = get_filtered_resized_dataset(dataset=ds_train, filter_label_value=194)
display_images_and_labels_from_dataset(some_filtered_ds)

In [ ]:
from tqdm.notebook import tqdm as ntqdm

def produce_TFRs_from_filtered_datasets(dataset, subset):
    for idx in ntqdm( range(N_Classes) ):
        label = str(idx).zfill(3)
        some_filtered_dataset = get_filtered_resized_dataset(dataset,filter_label_value=idx)
        dataset_to_TFR(some_filtered_dataset, prefix_target_filename=f'{subset}_{label}', features_keys=['img','label'])
        print(f'TFR for class {idx} finished!')

In [ ]:
produce_TFRs_from_filtered_datasets(dataset=ds_train, subset=f"{data_path}/train_L_")

In [ ]:
produce_TFRs_from_filtered_datasets(dataset=ds_valid, subset=f"{data_path}/valid_L_")

In [ ]:
produce_TFRs_from_filtered_datasets(dataset=ds_test, subset=f"{data_path}/test_L_")